# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


☁️ Colab rilevato.
Mounted at /content/drive
Cloning into 'MistakeDetection'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 408 (delta 98), reused 138 (delta 50), pack-reused 215 (from 1)
Receiving objects: 100% (408/408), 50.03 MiB | 28.98 MiB/s, done.
Resolving deltas: 100% (179/179), done.


# Dataset Setup

In [2]:
from utils import setup_project
# Ora puoi passare agli import del modello
from dataset.utils import SplitType
from dataset.capitain_cook_4d_mlp_dataset import CaptainCook4DMLP_Dataset, DatasetSource
from models.BaselineV1_MLP import BaselineV1_MLP

# Esegue: Setup Dati (unzip/copy), Login WandB, Setup Device
device = setup_project.initialize(ROOT_DIR)
# Import wandb
import wandb

Setup Progetto in: /content/MistakeDetection
source_path: /content/drive/MyDrive/MistakeDetection
Setup Dati da: /content/drive/MyDrive/MistakeDetection
Inizio setup dati...
   Sorgente: /content/drive/MyDrive/MistakeDetection
   Destinazione: /content/MistakeDetection/data
Estrazione ZIP: omnivore.zip...
Copia cartella: annotation_json...
Estrazione ZIP: slowfast.zip...
Estrazione ZIP: 3dresnet.zip...
Estrazione ZIP: x3d.zip...
Estrazione ZIP: omnivore_test.zip...
✅ Setup completato! Dati pronti in: /content/MistakeDetection/data


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s339450 (s339450-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB Logged in.
Device: cuda


# Configuration

In [3]:
# Configurazione esperimento
DATASET_SOURCE = DatasetSource.OMNIVORE
SPLIT_TYPE = SplitType.VIDEO_ID

config = {
    "architecture": "BaselineV1_MLP_" + DATASET_SOURCE.value + "_" + SPLIT_TYPE.value,
    "dataset": "CaptainCook4D",
    "feature_extractor": DATASET_SOURCE.value,
    "input_dim": DATASET_SOURCE.input_dims(),
    "batch_size": 512,
    "learning_rate": 0.001,
    "epochs": 50,
    "pos_weight": 1.5,
    "optimizer": "Adam",
    "loss_function": "BCEWithLogitsLoss",
    "seed": 42,
    "split_type": SPLIT_TYPE.value
}

# Dataset Split

In [4]:
import os
from dataset.capitain_cook_4d_mlp_dataset import CaptainCook4DMLP_Dataset, DatasetSource
from dataset.utils import get_mlp_loaders

try:
    full_dataset = CaptainCook4DMLP_Dataset(dataset_source=DATASET_SOURCE, root_dir=ROOT_DIR)
    train_loader, val_loader, test_loader = get_mlp_loaders(
        full_dataset,
        batch_size=config["batch_size"],
        seed=config["seed"],
        split_type=SPLIT_TYPE,
        test_ratio=0
    )

except Exception as e:
    print(f"❌ Errore: {e}")

Loading from: /content/MistakeDetection/data/omnivore...

DATASET INFO [V1 - SUBSECOND-BASED]
   Shape: torch.Size([283228, 1024]) -> 283228 Campioni, 1024 Features
FULL DATASET       | Tot: 283228 | OK: 198834 (70.2%) | ERR: 84394 (29.8%) | Ratio: 1:2.4
-------------------------------------------------------------------------------------
TRAIN SET          | Tot: 254839 | OK: 179043 (70.3%) | ERR: 75796 (29.7%) | Ratio: 1:2.4
VALIDATION SET     | Tot: 27881  | OK: 19283 (69.2%) | ERR: 8598  (30.8%) | Ratio: 1:2.2
TEST SET           | Tot: 508    | OK: 508   (100.0%) | ERR: 0     (0.0%)



In [5]:
full_dataset.print_item(0)

V1 DATASET ITEM [0]
Features shape:       torch.Size([1024]) (features)
Label:                0 (OK)
Step id:              115
Video id:             10_16


In [6]:
from utils.inspect_npz import inspect_npz_from_dataset

dataset_folder = DATASET_SOURCE.value
npz_filename = "1_7_360p.mp4_1s_1s.npz"

# Ispezione del file .npz
inspect_npz_from_dataset(full_dataset.features_dir(), npz_filename, n_rows=3)

File: /content/MistakeDetection/data/omnivore/1_7_360p.mp4_1s_1s.npz
Chiavi presenti nel file: ['arr_0']

Array 'arr_0' - shape: (604, 1024), dtype: float32
[[ 0.6910985   0.09298898 -0.6608225  ... -0.75679165  1.2401273
  -0.5683658 ]
 [ 0.40254688 -0.4466254  -0.8645446  ... -1.2709565   0.7917245
  -0.5052321 ]
 [ 0.643613   -0.48683766 -0.88651866 ... -1.0358062   0.658605
  -0.27201462]]


In [7]:
# Inizializzazione W&B
run = wandb.init(
    project="mistake-detection",
    name=f"baseline-mlp-v1-{DATASET_SOURCE.value}-{SPLIT_TYPE.value}",
    config=config,
    tags=["baseline", "mlp", DATASET_SOURCE.value],
    notes=f"Baseline MLP with {DATASET_SOURCE.value} features for mistake detection and {SPLIT_TYPE.value} split"
)

print(f"🚀 W&B Run: {run.name} (ID: {run.id})")

🚀 W&B Run: baseline-mlp-v1-omnivore-video_id (ID: anqau46g)


# W&B Configuration

# MLP (Version 1)

In [8]:
import torch
import torch.nn as nn

model = BaselineV1_MLP(DATASET_SOURCE.input_dims()).to(device)

# Watch del modello per tracciare gradienti e parametri
wandb.watch(model, log="all", log_freq=10)

In [9]:
lr = config["learning_rate"]
optimizer = torch.optim.Adam(model.parameters(), lr)

# Quanto pesa la classe "positiva" = classe "1" = classe "error":
# - CASO 1: rapporto effettivo del dataset
#train_pos_weight = train_cnt_0 / train_cnt_1

# - CASO 2: rapporto usato nel paper
train_pos_weight = config["pos_weight"]

print(f"Peso classe positiva: {train_pos_weight}")
train_pos_weight = torch.tensor([train_pos_weight], device=device)

criterion = nn.BCEWithLogitsLoss(pos_weight=train_pos_weight)

epochs = config["epochs"]

Peso classe positiva: 1.5


In [10]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import numpy as np

best_f1 = 0.0

for epoch in range(epochs):

    # -------------------------
    #        TRAIN
    # -------------------------
    model.train()
    total_loss = 0
    train_preds_list = []
    train_targets_list = []
    train_probs_list = []

    for inputs, labels, _, _ in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        outputs = model(inputs)            # [B, 1]
        outputs = outputs.squeeze(1)       # [B]

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metriche train
        with torch.no_grad():
            probs = torch.sigmoid(outputs).cpu()
            preds = (probs >= 0.5).long()

            train_preds_list.append(preds)
            train_targets_list.append(labels.cpu())
            train_probs_list.append(probs)

    avg_train_loss = total_loss / len(train_loader)

    # Metriche di training
    train_preds = torch.cat(train_preds_list).numpy()
    train_targets = torch.cat(train_targets_list).numpy()
    train_probs = torch.cat(train_probs_list).numpy()

    train_acc = accuracy_score(train_targets, train_preds)
    train_f1 = f1_score(train_targets, train_preds, zero_division=0)
    train_precision = precision_score(train_targets, train_preds, zero_division=0)
    train_recall = recall_score(train_targets, train_preds, zero_division=0)

    # AUC train (usa probabilità, NON predizioni)
    try:
        train_auc = roc_auc_score(train_targets, train_probs)
    except ValueError:
        train_auc = 0.0  # Caso raro con classe mancante nel batch

    # -------------------------
    #        EVAL
    # -------------------------
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels, _, _ in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs).squeeze(1)  # logits

            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

            probs = torch.sigmoid(outputs).cpu()
            preds = (probs >= 0.5).long()

            all_preds.append(preds)
            all_targets.append(labels.cpu())
            all_probs.append(probs)

    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()
    all_probs = torch.cat(all_probs).numpy()

    avg_val_loss = total_val_loss / len(val_loader)
    val_acc = accuracy_score(all_targets, all_preds)
    val_f1 = f1_score(all_targets, all_preds, zero_division=0)
    val_precision = precision_score(all_targets, all_preds, zero_division=0)
    val_recall = recall_score(all_targets, all_preds, zero_division=0)

    # AUC validation
    try:
        val_auc = roc_auc_score(all_targets, all_probs)
    except ValueError:
        val_auc = 0.0

    # Confusion Matrix
    cm = confusion_matrix(all_targets, all_preds)

    # Log su W&B
    wandb.log({
        # Training metrics
        "train/loss": avg_train_loss,
        "train/accuracy": train_acc,
        "train/f1": train_f1,
        "train/precision": train_precision,
        "train/recall": train_recall,
        "train/auc": train_auc,

        # Validation metrics
        "val/loss": avg_val_loss,
        "val/accuracy": val_acc,
        "val/f1": val_f1,
        "val/precision": val_precision,
        "val/recall": val_recall,
        "val/auc": val_auc,

        # Confusion Matrix
        "val/confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=all_targets,
            preds=all_preds,
            class_names=["No Error", "Error"]
        ),

        "learning_rate": optimizer.param_groups[0]['lr'],
        "epoch": epoch + 1
    })

    print(f"Epoch {epoch+1}/{epochs} "
          f"- Train Loss: {avg_train_loss:.4f} "
          f"- Val Loss: {avg_val_loss:.4f} "
          f"- Val Acc: {val_acc:.4f} "
          f"- Val F1: {val_f1:.4f} "
          f"- Val Precision: {val_precision:.4f} "
          f"- Val Recall: {val_recall:.4f} "
          f"- Val AUC: {val_auc:.4f}")

    # Salvataggio miglior modello
    if val_f1 > best_f1:
        best_f1 = val_f1
        checkpoint_path = os.path.join(ROOT_DIR, "checkpoints", f"best_model_f1_{best_f1:.4f}.pth")
        os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_f1': val_f1,
            'val_acc': val_acc,
            'val_auc': val_auc,
        }, checkpoint_path)

        artifact = wandb.Artifact(
            name=f"model-{run.id}",
            type="model",
            description=f"Best model with F1={best_f1:.4f}",
            metadata={
                "epoch": epoch + 1,
                "val_f1": val_f1,
                "val_acc": val_acc,
                "val_auc": val_auc,
                "architecture": config["architecture"]
            }
        )
        artifact.add_file(checkpoint_path)
        wandb.log_artifact(artifact)

        print(f"✅ Nuovo miglior modello salvato! F1: {best_f1:.4f}")

print("\n🎉 Training completato!")
print(f"Miglior F1 Score: {best_f1:.4f}")

Epoch 1/50 - Train Loss: 0.6417 - Val Loss: 0.8488 - Val Acc: 0.6689 - Val F1: 0.3878 - Val Precision: 0.4511 - Val Recall: 0.3401 - Val AUC: 0.6327
✅ Nuovo miglior modello salvato! F1: 0.3878
Epoch 2/50 - Train Loss: 0.5264 - Val Loss: 0.9058 - Val Acc: 0.6805 - Val F1: 0.3446 - Val Precision: 0.4689 - Val Recall: 0.2724 - Val AUC: 0.6332
Epoch 3/50 - Train Loss: 0.4695 - Val Loss: 0.9730 - Val Acc: 0.6726 - Val F1: 0.3822 - Val Precision: 0.4570 - Val Recall: 0.3284 - Val AUC: 0.6386
Epoch 4/50 - Train Loss: 0.4307 - Val Loss: 1.0798 - Val Acc: 0.6615 - Val F1: 0.3962 - Val Precision: 0.4403 - Val Recall: 0.3602 - Val AUC: 0.6277
✅ Nuovo miglior modello salvato! F1: 0.3962
Epoch 5/50 - Train Loss: 0.4032 - Val Loss: 1.0776 - Val Acc: 0.6710 - Val F1: 0.3737 - Val Precision: 0.4524 - Val Recall: 0.3183 - Val AUC: 0.6300
Epoch 6/50 - Train Loss: 0.3829 - Val Loss: 1.0596 - Val Acc: 0.6630 - Val F1: 0.3970 - Val Precision: 0.4428 - Val Recall: 0.3599 - Val AUC: 0.6327
✅ Nuovo miglior mo

# Results & Visualization

In [ ]:
# Log della tabella con esempi di predizioni
model.eval()
predictions_table = wandb.Table(
    columns=["ID", "True Label", "Predicted", "Probability", "Correct"]
)

global_id = 0

with torch.no_grad():
    for inputs, labels, _, _ in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        outputs = model(inputs).squeeze(1)
        probs = torch.sigmoid(outputs)
        preds = (probs >= 0.5).long()

        for i in range(min(50, len(labels))):
            predictions_table.add_data(
                global_id,
                labels[i].item(),
                preds[i].item(),
                probs[i].item(),
                preds[i].item() == labels[i].item()
            )
            global_id += 1

        break  # solo un batch

wandb.log({"predictions/sample_table": predictions_table})

# Summary
wandb.run.summary["best_f1"] = best_f1
wandb.run.summary["final_val_acc"] = val_acc
wandb.run.summary["final_val_precision"] = val_precision
wandb.run.summary["final_val_recall"] = val_recall

print("✅ Tabella predizioni e summary caricati su W&B")

✅ Tabella predizioni e summary caricati su W&B


In [ ]:
# Chiudi il run di W&B
wandb.finish()
print("🏁 W&B run terminato")

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,▁▂▃▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████
train/f1,▁▂▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
train/loss,█▇▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train/precision,▁▂▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████
train/recall,▁▂▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
val/accuracy,▃▄▅▁▄▅▆▄▅▅▅▅▆▆▅▇▇▆▇▆▇▆▆▆▆▇▆▇▇▇▇▇█▇▆▇█▇▇█
val/f1,▁▂▅▅▅▅▆▅▆▆▃▆▆▆▇▆▅██▇▇▅▆▆▇▇▆▆▆▆▆▆▆▆▆▆▆▇▆▅
val/loss,▁▁▁▂▂▂▂▂▃▃▃▃▄▃▄▄▄▄▅▄▅▅▅▅▆▅▆▆▆▆▇▇▇▇▇▇▇▇▇█
+2,...


🏁 W&B run terminato
